**FGV EPGE - Escola de Pós-graduação em Economia**
==
**Trabalho da Disciplina Machine Learning**
==
Professores: Genaro Lins e Rafael Martins de Souza


Monitora: Maria Gabriela Gontijo

**Tema: Predição de Default com Técnicas de Machine Learning**
==
Alunos: Marlon Marques Almeida e Rodrigo Ferreira Saraiva da Fonseca

Data: 17/12/2021


#Objetivo

O objetivo deste trabalho é apresentar e comparar algumas técnicas de machine learning para classificação supervisionada.

Será utilizada uma base com dados socio-econômicos para predição da probabilidade de default de cartão de crédito.

As instituições financeiras tem como base do seu negócio captação de recursos para aplicação dos mesmos aos tomadores. Esse ciclo econônico é de extrema importância para atividade econômica. No entanto, quando temos alto grau de inadimplência, esse ciclo pode ficar comprometido, impactando toda economia. 

As instituições financeiras têm investido cada vez mais em análise de dados e modelos para predição de propabilidade de defaut, buscando redução dessa indadimplência. Com o advento do Open Banking no Brasil, a minimização da ocorrência de defaults torna-se ainda mais importante para manutenção e crescimento de uma instiuição. 

No trabalho, utilizamos uma base de dados da American Express de usuários dos EUA objetivando uma classificação da probabilidade de default.

A título de curiosidade, abaixo colocamos volume de concessão de limites de cartões crédito na pessoa física no Brasil nos últimos anos e a indimplência nessa linha de crédito. 

In [ ]:
import pandas as pd
#Declaração do código da série de interesse -  Concessões - Pessoas físicas - Cartão de crédito total
codigo = 20682
#Declaração da URLInadimplência da carteira - Pessoas físicas - Cartão de crédito total
codigo2 = 21129
#Declaração da URL
url = 'http://api.bcb.gov.br/dados/serie/bcdata.sgs.{}/dados?formato=json'.format(codigo)
url2 = 'http://api.bcb.gov.br/dados/serie/bcdata.sgs.{}/dados?formato=json'.format(codigo2)

concessaocartaocreidopf = pd.read_json(url)
inadimplencia = pd.read_json(url2)
concessaocartaocreidopf['data'] = pd.to_datetime(concessaocartaocreidopf['data'], format = '%d/%m/%Y')
inadimplencia['data'] = pd.to_datetime(inadimplencia['data'], format = '%d/%m/%Y')


: 

In [ ]:
#Construíndo Gráficos das Variáveis no Tempo 
import seaborn as sns
g1 = sns.relplot(data=inadimplencia, x="data", y="valor",kind="line", palette="crest", linewidth=4,height=2, aspect=8, legend=True)
g2 = sns.relplot(data=concessaocartaocreidopf, x="data", y="valor",kind="line", palette="crest", linewidth=4,height=2, aspect=8, legend=True)


: 

In [ ]:
inadimplencia.tail()

: 

In [ ]:
concessaocartaocreidopf.tail()

: 

# Default

Defaut pode ser entendido como não pagamento de alguma obrigação, trazendo prejuízo para pessoas físicas, instituições financeiras, empresas e para economia como um todo.

Existem vários estudos e algoritimos para análise de padrões na concessão de crédito, principalmente em grandes instuições, visando reduzir o impacto da inadimplência em sua carteira de clientes. 

*Mariane Daros e Nelson Guilherme Machado Pinto* colocam que: "Nenhuma empresa deseja ter inadimplentes em sua carteira de clientes. Assim, conhecer o perfil de seus atuais clientes e compreender as causas dos inadimplentes são alguns dos meios para evitar tal fenômeno".

Olhando para lado da pessoa física, existem inúmeros fenônemos que podem contribuir para o aumento da inadimplência, um exemplo em evidência nos últimos ano é a pandemia da COVID-19.

Normalmente ela ocorre por diversos fatores, desde redução na renda até esquecimento do vencimento da dívida.

Quando vamos para Cenário da Inadimplência na Pessoa Jurídica, também observamos inúmeros fatores que contribuem para tal, como redução de vendas, aumento de impostos ou taxa juros, dentre muitos outros.

Cabe destacar que inadimplência como consequência desses fatores podem levar  a crises ecônomicas enormes, como foi, por exemplo, a crise imobiliária nos EUA em 2009, que impactou o mundo tudo.


#Base de dados

A base de dados utilizada está disponível no Kaggle e foi utilizada na competição AmExpert 2021 CODELAB – Machine Learning Hackathon.

https://www.kaggle.com/pradip11/amexpert-codelab-2021

Configurações iniciais

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from google.colab import drive 
drive.mount('/content/drive')
np.random.seed(42)

: 

##Importando a base de dados

In [ ]:
#Importando do drive do Marlon
df=pd.read_csv('/content/drive/MyDrive/PROJETO-FINAL-MACHINE-LEARNING/train.csv')

: 

In [ ]:
#Importando do drive do Rodrigo
#os.chdir('/content/drive/MyDrive/Colab Notebooks/FGV MFEE 2021/Machine Learning/Trabalho Final')
#df=pd.read_csv('train.csv')

: 

##Verificando a base de dados

In [ ]:
df.head()

: 

Descrição das variáveis:


**customer_id** -	Represents the unique identification of a customer

**name** - 	Represents the name of a customer

**age** -	Represents the age of a customer ( in years ) 

**gender** -	Represents the gender of a customer( F means Female and M means Male  )

**owns_car** -	Represents whether a customer owns a car ( Y means Yes and N means No  )

**owns_house** - Represents whether a customer owns a house ( Y means Yes and N means No  )

**no_of_children**	- Represents the number of children of a customer

**net_yearly_income**	- Represents the net yearly income of a customer ( in USD )

**no_of_days_employed**	- Represents the no of days employed

**occupation_type**	- Represents the occupation type of a customer ( IT staff, Managers, Accountants, Cooking staff, etc )

**total_family_members**	- Represents the number of family members of a customer

**migrant_worker** -	Represents whether a customer is a migrant worker( 1 means Yes and 0 means No  ) 

**yearly_debt_payments** -	Represents the yearly debt payments of a customer  ( in USD )

**credit_limit**	- Represents the credit limit of a customer  ( in USD ) 

**credit_limit_used(%)**	- Represents the percentage of credit limit used by a customer

**credit_score** -	Represents the credit score of a customer

**prev_defaults**	- Represents the number of previous defaults

**default_in_last_6months** -	Represents whether a customer has defaulted in the last 6 months ( 1 means Yes and 0 means No) 

**credit_card_default**	- Represents whether there will be credit card default  ( 1 means Yes and 0 means No  ) 

Temos ao todo 19 colunas. As colunas customer_id e name não serão relevantes para o algoritmo de machine learning. A coluna credit_card_defautl é o que queremos estimar, então será o nosso label. 

Restam, portanto, 16 características. Destas 16 características, 4 (gender, owns_car, owns_house e occupation_type) são categóricas e 12 numéricas.

Podemos observar que a base contém 45528 instâncias e há alguns missing values nas características owns_car, no_of_children, no_of_days_employed, total_family_members, migrant_worker, yearly_debt_payments e credit_score.

In [ ]:
df.info()

: 

As escalas das variáveis são batante variadas, principalmente em função da presença de um número grande de varíaveis discretas e binárias. (ex: os valores máximos de  net_yearly_income e total_family_members são, respectivamente, 1.407590e+08	e 10).

In [ ]:
df.describe()

: 

In [ ]:
df.hist(bins=50, figsize=(20,15))
plt.show()

: 

Além disso, podemos constantar que a base de dados é desbalanceada, pois a nossa varíavel alvo, quando poussui falar positivo, tem pouca representatividade na base (<10%).

In [ ]:
default_prop = df["credit_card_default"].value_counts()/len(df)
default_prop

: 

##Avaliando possíveis estratificações da base

###Gênero

Temos 3 tipos de gênero na base de dados (F, M e XNA). No entanto, XNA possui apenas uma observação, o que inviabiliza o seu uso no modelo.

Identificamos as seguintes opções de tratamento:

1.   Atribuí-la arbitrariamente a algum grupo;
2.   Expurgar da base.

Dado que estamos falando de 1 instância em 45k+, o impacto é irrelevante em quaisquer dos grupos. Portanto optamos por retirar da base.

In [ ]:
df["gender"].value_counts()

: 

Excluindo a linha com gender = XNA

In [ ]:
df.drop(df[df['gender']=='XNA'].index,inplace=True)

: 

In [ ]:
df["gender"].value_counts()

: 

Verificando a proporção dos sexos na base de dados.

In [ ]:
gender_prop = df["gender"].value_counts()/len(df)
gender_prop

: 

Verificando como a variável alvo (credit_card_default) está distribuída nos sexos.

In [ ]:
gender_default_prop = df[df['credit_card_default']==1]['gender'].value_counts()/df["gender"].value_counts()
gender_default_prop

: 

Podemos constatar que a proporção de default é maior no sexo masculino que no sexo feminino.

In [ ]:
gender_resume = pd.concat([gender_prop, gender_default_prop], axis=1)*100
gender_resume.columns = ['% de Instâncias', '% de Default']
gender_resume

: 

In [ ]:
#plt.barh(y = occ_prop.index, width=occ_prop_inc.values, label='Proporção do Total de Renda')
#plt.xlabel("Proporção")
gender_resume.plot.bar(figsize=(18,9))
plt.title('Proporção de Gender e Default', fontweight="bold")
plt.legend(loc="upper right")
plt.show()

: 

###Credit Score

In [ ]:
#Criei uma cópia da base de dados original para utilização na parte de estratificação por credit score
df2 = df.copy()

: 

In [ ]:
df['credit_score'].unique()

: 

Como já vimos que credit_score possui missing values, precisaremos eliminá-los para fazer a estratificação com base nessa característica.

In [ ]:
df.drop(df[df['credit_score'].isna()].index, inplace=True)
df['credit_score'].isna().sum()

: 

In [ ]:
df['cred_score_cat'] = pd.cut(df['credit_score'], bins=([0, 700, 850, np.inf]), labels=['<700', '<850', '850+'])
df.groupby('cred_score_cat')['customer_id'].count()/len(df)

: 

In [ ]:
df['cred_score_cat']

: 

Conferindo a coluna cred_score_cat


In [ ]:
df.head()

: 

Combinando Gender e Credt Score

In [ ]:
df['gender_cred_score_cat'] = df['gender'].astype(str) + df['cred_score_cat'].astype(str)
df['gender_cred_score_cat']

: 

In [ ]:
gender_cred_score_prop = df.groupby('gender_cred_score_cat')['customer_id'].count()/len(df)
gender_cred_score_prop

: 

"Dropando" cred_score_cat, pois não precisamos mais dela.

In [ ]:
df.drop('cred_score_cat', axis=1, inplace=True)
df.head()

: 

#Separando as bases de treino e de teste

Realizando a separação das bases considerando a estratificação apenas do sexo.

<b><font color=blue> Enquanto não fechamos a questão da estratificação por faixa etária, vamos excluir a coluna age_cat para podermos seguir comn o código.



In [ ]:
df = df.reset_index()
df.head()

: 

In [ ]:
from sklearn.model_selection import StratifiedShuffleSplit

split = StratifiedShuffleSplit(n_splits=1, test_size=0.2)
for train_index, test_index in split.split(df, df["gender_cred_score_cat"]):
    train_set = df.loc[train_index]
    test_set = df.loc[test_index]

: 

Verificando o tamanho dos conjuntos criados.

In [ ]:
print(f"Treino: {len(train_set)}\nTeste: {len(test_set)}")

: 

Conferindo se a estratificação foi executada corretamente.

In [ ]:
gender_cred_score_prop_train = train_set.groupby('gender_cred_score_cat')['customer_id'].count()/len(train_set)
gender_cred_score_prop_test = test_set.groupby('gender_cred_score_cat')['customer_id'].count()/len(test_set)

: 

In [ ]:
compara_props_strat = pd.concat([gender_cred_score_prop, gender_cred_score_prop_test, gender_cred_score_prop_test],axis=1)
compara_props_strat.columns = ['Prop DF', 'Prop Train Set', 'Prop Test Set']
compara_props_strat

: 

Excluindo a coluna de estratificação das bases de treino e teste.

In [ ]:
#Excluindo da base de treino e teste atraibuto Credit Score Cat
train_set.drop('gender_cred_score_cat', axis=1, inplace=True)
test_set.drop('gender_cred_score_cat', axis=1, inplace=True)

: 

In [ ]:
train_set.columns

: 

In [ ]:
test_set.columns

: 

#Explorando os dados

Criando uma cópia do conjunto de treinamento para preservar o original.

In [ ]:
default = train_set.copy()
default.head()

: 

Aparentemente há uma correlação positiva e alta do label (credit_card_default) com as características default_in_last_6months e prev_defaults.

Com relação às correlações negativas, destacam-se no_of_days_employed e o credit_score, o que está dentro do esperado.

In [ ]:
corr_matrix = default.corr()
corr_matrix['credit_card_default'].sort_values(ascending=False)

: 

In [ ]:
from pandas.plotting import scatter_matrix

attributes = ["credit_card_default", "default_in_last_6months", "prev_defaults",
              "credit_limit_used(%)", 'no_of_days_employed', 'credit_score']
scatter_matrix(default[attributes], figsize=(16, 12))
plt.show()

: 

#Preparando os dados para os algoritmos

##Separando características e labels

Características.

In [ ]:
todrop = ['credit_card_default', 'index']
default_X = train_set.drop(todrop, axis=1)
default_X.head()

: 

Labels.

In [ ]:
default_Y = train_set["credit_card_default"].copy()
default_Y.head()

: 

##Definindo e separando as variáveis numéricas e categóricas

###Variáveis categóricas

Separando variáveis categóricas.

In [ ]:
default_X.head()

: 

In [ ]:
cat = ['gender', 'owns_car', 'owns_house', 'occupation_type', 'migrant_worker']
cat

: 

In [ ]:
default_X_cat = default_X[cat]
default_X_cat

: 

Temos 19 tipos de ocupação.




In [ ]:
default_X['occupation_type'].value_counts()

: 

###Variáveis numéricas

Separando variáveis numéricas.

In [ ]:
num = ['age', 'no_of_children', 'net_yearly_income', 'no_of_days_employed' ,
       'total_family_members', 'yearly_debt_payments', 'credit_limit',
       'credit_limit_used(%)', 'credit_score', 'prev_defaults', 'default_in_last_6months']
num

: 

In [ ]:
default_X_num = default_X[num]
default_X_num.head()

: 

##Preparando os pipelines de transformação dos dados

###Pipeline numérico

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

: 

Definindo o pipeline numérico

In [ ]:
num_pipeline = Pipeline([
                         ('imputer', SimpleImputer(strategy='median')),
                         ('std_scaler', StandardScaler())
])

: 

Aplicando o pipeline numérico.

In [ ]:
default_X_num_prep = num_pipeline.fit_transform(default_X_num)
default_X_num_prep

: 

In [ ]:
default_X_num_prep.shape

: 

Verificando o resultado da transformação.

In [ ]:
pd.DataFrame(default_X_num_prep, columns=num).describe()

: 

In [ ]:
pd.DataFrame(default_X_num_prep, columns=num).hist(bins=50, figsize=(20,15))
plt.show

: 

###Pipeline categórico

In [ ]:
default_X_cat

: 

Definindo o encoder categórico.

In [ ]:
from sklearn.preprocessing import OneHotEncoder
cat_encoder = OneHotEncoder()

: 

In [ ]:
default_X_cat_1hot = cat_encoder.fit_transform(default_X_cat)
default_X_cat_1hot = default_X_cat_1hot.toarray()
default_X_cat_1hot

: 

Temos 2 colunas novas a mais do que esperado, pois owns_car e migrant_worker têm missing values, então o OneHotEncoder gerou uma coluna a mais para cada um deles para representar o missing value. 

Como não é possível arbitrar um valor da mesma forma que fazemos para variáveis números, temos as seguintes alternativas: 

1.   Ignorar as observações;
2.   Ignorar a variável;
3.   Desenvolver um modelo para predizer os *missing values*;
4.   Tratar como se fosse mais uma categoria.

Entendemos que no nosso caso a 4ª opção seria a mais interessante e seguimos com ela.






In [ ]:
default_X_cat.nunique()

: 

In [ ]:
default_X_cat.nunique().sum()

: 

In [ ]:
default_X_cat.isna().sum()

: 

In [ ]:
default_X_cat_1hot.shape

: 

###Pipeline completo

In [ ]:
from sklearn.compose import ColumnTransformer

: 

In [ ]:
full_pipeline = ColumnTransformer([
                                   ("num", num_pipeline, num),
                                   ("cat",OneHotEncoder(), cat)
])

: 

In [ ]:
default_X_prep = full_pipeline.fit_transform(default_X)

: 

In [ ]:
default_X_prep.shape

: 

Preparando a base de teste.

In [ ]:
todrop = ['credit_card_default', 'index']
default_X_test = test_set.drop(todrop, axis=1)
default_X_test.head()

: 

In [ ]:
default_Y_test = test_set["credit_card_default"].copy()
default_Y_test.head()

: 

In [ ]:
default_X_test_prep = full_pipeline.transform(default_X_test)
default_X_test_prep

: 

#Treinando modelos

**Modelos que serão treinados:**


1.   Regressão Logística
2.   Bagging
3.   Árvore de Decisão
4.   Random Forest
5.   Gradient Boosting




In [ ]:
from sklearn.model_selection import cross_val_predict

: 

##Definindo algumas funções de score

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score
from sklearn.model_selection import cross_val_score

: 

In [ ]:
#Função para retornar matriz de confusão, precision, recall e f1
def score_completo(Y, Y_PRED):

  conf_mx = confusion_matrix(Y, Y_PRED)
  conf_mx = pd.DataFrame(conf_mx, columns=['Negativo Previsto', 'Positivo Previsto'], index=['Negativo Real', 'Positivo Real'])
  accuracy = round(accuracy_score(Y, Y_PRED),4)
  precision = round(precision_score(Y, Y_PRED),4)
  recall = round(recall_score(Y, Y_PRED),4)
  f1 = round(f1_score(Y, Y_PRED),4)

  return print(f'                    \033[1mConfusion Matrix\033[0m \n{conf_mx}\n\n Accuracy: {accuracy} | Precision: {precision} | Recall: {recall} | F1: {f1}')

: 

In [ ]:
#Função para retornar os valores de accuracy, precision, recall e f1
def resumo_score(Y, Y_PRED):

  accuracy = round(accuracy_score(Y, Y_PRED),4)
  precision = round(precision_score(Y, Y_PRED),4)
  recall = round(recall_score(Y, Y_PRED),4)
  f1 = round(f1_score(Y, Y_PRED),4)

  return accuracy, precision, recall, f1

: 

In [ ]:
#função para calcular o cross validation para as métricas accuracy, precision, recall e f1
def resumo_xval_score(model, X, Y, kfold):

  accuracy = cross_val_score(model, X, Y, cv=kfold, scoring="accuracy")
  precision = cross_val_score(model, X, Y, cv=kfold, scoring="precision")
  recall = cross_val_score(model, X, Y, cv=kfold, scoring="recall")
  f1 = cross_val_score(model, X, Y, cv=kfold, scoring="f1")

  return accuracy.mean(), precision.mean(), recall.mean(), f1.mean()

: 

##Regressão Logística

In [ ]:
from sklearn.linear_model import LogisticRegression

: 

200 iterações foi o suficiente para convergência (com 100 deu erro no fit e 150 no cross_validation).

In [ ]:
#Definindo o modelo
lr = LogisticRegression(max_iter=200)

: 

**Base de Treino**

In [ ]:
lr.fit(default_X_prep, default_Y)

: 

In [ ]:
#Predizendo os valores no conjunto de treino
pred_train_lr = lr.predict(default_X_prep)
pred_train_lr

: 

In [ ]:
#Score sem validação cruzada
score_completo(default_Y, pred_train_lr)

: 

In [ ]:
#Predição com validação cruzada
xpred_train_lr = cross_val_predict(lr, default_X_prep, default_Y, cv=5)

: 

In [ ]:
#Score com validação cruzada
score_completo(default_Y, xpred_train_lr)

: 

**Base de Teste**

In [ ]:
#Predição sem validação cruzada
pred_test_lr = lr.predict(default_X_test_prep)

: 

In [ ]:
#Score sem validação cruzada
score_completo(default_Y_test, pred_test_lr)

: 

In [ ]:
#Predição com validação cruzada
xpred_test_lr = cross_val_predict(lr, default_X_test_prep, default_Y_test, cv=5)

: 

In [ ]:
#Score Com validação cruzada
score_completo(default_Y_test, xpred_test_lr)

: 

##Regressão Logística com Bagging

In [ ]:
from sklearn.ensemble import BaggingClassifier

: 

###Modelo antes do GridSearch

In [ ]:
#parâmetros default = n_estimators: default=10; bootstrap: default=True; bootstrap_features: default=False
#parâmetros 1ª tentativa = lr, n_estimators=500, bootstrap=False
bag_lr = BaggingClassifier(lr)

: 

**Base de Treino**

In [ ]:
bag_lr.fit(default_X_prep, default_Y)

: 

In [ ]:
#Predição sem validação cruzada
pred_train_bag_lr = bag_lr.predict(default_X_prep)
pred_train_bag_lr

: 

In [ ]:
#Score sem validação cruzada
score_completo(default_Y, pred_train_bag_lr)

: 

In [ ]:
#Predição com validação cruzada
xpred_train_bag_lr = cross_val_predict(bag_lr, default_X_prep, default_Y, cv=5)
xpred_train_bag_lr

: 

In [ ]:
#Score com validação cruzada
score_completo(default_Y, xpred_train_bag_lr)

: 

**Base de Teste**

In [ ]:
#Predição sem validação cruzada
pred_test_bag_lr = bag_lr.predict(default_X_test_prep)
pred_test_bag_lr

: 

In [ ]:
#Score sem validação cruzada
score_completo(default_Y_test, pred_test_bag_lr)

: 

In [ ]:
#Predição com validação cruzada
xpred_test_bag_lr = cross_val_predict(bag_lr, default_X_test_prep, default_Y_test, cv=5)
xpred_test_bag_lr

: 

In [ ]:
#Score com validação cruzada
score_completo(default_Y_test, xpred_test_bag_lr)

: 

<b><font color=black>Podemos observar que os valores foram bem próximos ao realizado pelo conjunto de treinamento.



###Modelo após GridSearch

In [ ]:
#Rodei Grid Search para Otimização
from sklearn.ensemble import BaggingClassifier
from sklearn.model_selection import GridSearchCV

param_grid1 = [
    {'n_estimators': [3, 10, 30], 'max_features': [2, 4,8,16]},
    {'bootstrap': [False], 'n_estimators': [3, 10], 'max_features': [2, 3, 4]},
  ]

bag_lr1 = BaggingClassifier(random_state=42)

grid_search_reg_bag = GridSearchCV(bag_lr1, param_grid1, cv=5,
                           return_train_score=True)
grid_search_reg_bag.fit(default_X_prep, default_Y)

: 

In [ ]:
grid_search_reg_bag.best_params_

: 

In [ ]:
grid_search_reg_bag.best_estimator_


: 

In [ ]:
bag_lr_2 = BaggingClassifier(n_estimators=30, max_features=16)
bag_lr_2.fit(default_X_prep, default_Y)

: 

In [ ]:
#Predição com validação cruzada com parâmetros estimados pelo Grid Search
cross_pred_train_bag_lr_2 = cross_val_predict(bag_lr_2, default_X_prep, default_Y, cv=5)
cross_pred_train_bag_lr_2

: 

In [ ]:
score_completo(default_Y, cross_pred_train_bag_lr_2)

: 

In [ ]:
cross_pred_test_bag_lr_2 = cross_val_predict(bag_lr_2, default_X_test_prep, default_Y_test, cv=5)
cross_pred_test_bag_lr_2

: 

In [ ]:
score_completo(default_Y_test, cross_pred_test_bag_lr_2)

: 

##Árvore de Decisão

In [ ]:
from sklearn.tree import DecisionTreeClassifier

: 

###Modelo antes do GridSearch

In [ ]:
#parâmetros padrão - max_depth: default=None; max_features: default=None; max_leaf_nodes: default=None
tree_clf = DecisionTreeClassifier()

: 

**Base de Treino**

In [ ]:
tree_clf.fit(default_X_prep, default_Y)

: 

In [ ]:
tree_clf.get_depth()

: 

In [ ]:
tree_clf.get_n_leaves()

: 

In [ ]:
#Predição sem validação cruzada
pred_train_tree_Y = tree_clf.predict(default_X_prep)
pred_train_tree_Y

: 

In [ ]:
#Score sem validação cruzada
score_completo(default_Y, pred_train_tree_Y)

: 

In [ ]:
#Predição com validação cruzada
cross_pred_train_tree_Y = cross_val_predict(tree_clf, default_X_prep, default_Y, cv=5)
cross_pred_train_tree_Y

: 

In [ ]:
#Score com validação cruzada
score_completo(default_Y, cross_pred_train_tree_Y)

: 

**Base de Teste**

In [ ]:
#Predição sem validação cruzada
pred_test_tree_clf_Y = tree_clf.predict(default_X_test_prep)
pred_test_tree_clf_Y

: 

In [ ]:
#score sem validação cruzada
score_completo(default_Y_test, pred_test_tree_clf_Y)

: 

In [ ]:
#Predição com validação cruzada
xpred_test_tree_clf = cross_val_predict(tree_clf, default_X_test_prep, default_Y_test, cv=5)
xpred_test_tree_clf

: 

In [ ]:
#score com validação cruzada
score_completo(default_Y_test, xpred_test_tree_clf)

: 

###Modelo após GridSearch

In [ ]:
#Rodei Grid Search para Otimização
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV

param_grid2 = [
    { 'max_depth': [3, 10, 30],'max_features': [None],'max_leaf_nodes': [8,32, 64, 128]},
     ]

tree_clf_2 = DecisionTreeClassifier(random_state=42)

grid_search_tree = GridSearchCV(tree_clf_2, param_grid2, cv=5,
                           return_train_score=True)
grid_search_tree.fit(default_X_prep, default_Y)

: 

<b><font color=red> Tentamos usar Max_features com valores porém modelo estava entregando resultado do recall e R1 muito baixo, por isso usamos None.

In [ ]:
grid_search_tree.best_params_

: 

In [ ]:
grid_search_tree.best_estimator_

: 

In [ ]:
tree_clf3 = DecisionTreeClassifier(max_depth=3,max_features=None, max_leaf_nodes=8)
tree_clf3.fit(default_X_prep, default_Y)

: 

In [ ]:
#Predição com validação cruzada com parâmetros estimados pelo Grid Search
cross_pred_train_tree_clf_3 = cross_val_predict(tree_clf3, default_X_prep, default_Y, cv=5)
cross_pred_train_tree_clf_3

: 

In [ ]:
#Score com validação cruzada
score_completo(default_Y, cross_pred_train_tree_clf_3)

: 

**Base de Teste**

In [ ]:
#Predição com validação cruzada
xpred_test_tree_clf_3 = cross_val_predict(tree_clf3, default_X_test_prep, default_Y_test, cv=5)
xpred_test_tree_clf_3

: 

In [ ]:
#score com validação cruzada
score_completo(default_Y_test, xpred_test_tree_clf_3)

: 

##Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

: 

###Modelo antes do GridSearch

**Base de Treino**

In [ ]:
#parâmetros padrão -  n_estimators: default=100; max_depth: default=None; max_features{“auto”, “sqrt”, “log2”}: default=”auto”; max_leaf_nodes: default=None; bootstrap: default=True
rndf_clf = RandomForestClassifier()
rndf_clf.fit(default_X_prep, default_Y)

: 

In [ ]:
#Predição sem validação cruzada
pred_train_rndf_clf = rndf_clf.predict(default_X_prep)
pred_train_rndf_clf

: 

In [ ]:
#Score com predição sem validação cruzada
score_completo(default_Y, pred_train_rndf_clf)

: 

In [ ]:
#Predição com validação cruzada
xpred_train_rndf_clf = cross_val_predict(rndf_clf, default_X_prep, default_Y, cv=5)
xpred_train_rndf_clf

: 

In [ ]:
#Score com predição com validação cruzada
score_completo(default_Y, xpred_train_rndf_clf)

: 

**Base de Teste**

In [ ]:
#Predição sem validação cruzada
pred_test_rndf_clf = rndf_clf.predict(default_X_test_prep)
pred_test_rndf_clf

: 

In [ ]:
#Score com predição sem validação cruzada
score_completo(default_Y_test, pred_test_rndf_clf)

: 

In [ ]:
#Predição com validação cruzada
xpred_test_rndf_clf = cross_val_predict(rndf_clf, default_X_test_prep, default_Y_test, cv=5)
xpred_test_rndf_clf

: 

In [ ]:
#Score com predição com validação cruzada
score_completo(default_Y_test, xpred_test_rndf_clf)

: 

###Modelo após GridSearch

In [ ]:
#Rodei Grid Search para Otimização

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

param_grid3 = [

    {'n_estimators': [3, 10, 30], 'max_features': [2, 4, 8, 16], 'max_leaf_nodes': [8, 32, 64, 128]},

    {'bootstrap': [False], 'n_estimators': [3, 10], 'max_features': [2, 3, 4],'max_leaf_nodes': [8, 16, 32]},
  ]

rndf_clf_gscv = RandomForestClassifier(random_state=42)

grid_search3 = GridSearchCV(rndf_clf_gscv, param_grid3, cv=5,
                           return_train_score=True)
grid_search3.fit(default_X_prep, default_Y)

: 

In [ ]:
grid_search3.best_params_

: 

In [ ]:
grid_search3.best_estimator_

: 

**Base de Treino**

In [ ]:
#A título de curiosidade, vamos rodar Random Florest com n_estimators=500, max_leaf_nodes=16 e max_features=6 e  e comparar com os parametros estimados pelo GridSearch
rndf_clf_1 = RandomForestClassifier(n_estimators=500, max_leaf_nodes=16, max_features=6)
rndf_clf_1.fit(default_X_prep, default_Y)

: 

In [ ]:
#Agora Aplicaremos Valores Encontrados no GridSearch
rndf_clf_2 = RandomForestClassifier(n_estimators=30 ,max_leaf_nodes=32 , max_features = 16)
rndf_clf_2.fit(default_X_prep, default_Y)

: 

In [ ]:
#Predição sem validação cruzada
pred_train_rndf_Y = rndf_clf_2.predict(default_X_prep)
pred_train_rndf_Y

: 

In [ ]:
#Predição com validação cruzada com parâmetros estimados pelo Grid Search
cross_pred_train_rndf_clf_2 = cross_val_predict(rndf_clf_2, default_X_prep, default_Y, cv=5)
cross_pred_train_rndf_clf_2

: 

In [ ]:
#Score com predição com validação cruzada para parâmetros do Grid Search
#Encontramos valores iguais para Accuracy, Recall e bem próximos para Precision e F1, com max_leaf_nodes e  n_estimators bem menores que nos estimadores aleatórios.
score_completo(default_Y, cross_pred_train_rndf_clf_2)

: 

<b><font color=green> Deixei duas comparações acima para mostrar que Grid Search propoe valores com resultados bem próximos e número menor de valores, o que gera ganho de tempo ao rodar 

**Base de Teste**

In [ ]:
pred_test_rndf_Y = rndf_clf_2.predict(default_X_test_prep)
pred_test_rndf_Y

: 

In [ ]:
score_completo(default_Y_test, pred_test_rndf_Y)

: 

In [ ]:
cross_pred_test_rndf_clf_2 = cross_val_predict(rndf_clf_2, default_X_test_prep, default_Y_test, cv=5)
cross_pred_test_rndf_clf_2

: 

In [ ]:
score_completo(default_Y_test, cross_pred_test_rndf_clf_2)

: 

##Gradient Boosting

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

: 

###Modelo antes do GridSearch

**Base de Treino**

In [ ]:
#parâmetros padrão - learning_rate: default=0.1; n_estimators: default=100; max_depth: default=3; 
#max_features{‘auto’, ‘sqrt’, ‘log2’}: default=None; max_leaf_nodes: default=None; tol: default=1e-4
#parâmetros da primeira tentativa para referência n_estimators=500, max_leaf_nodes=16
gb_clf = GradientBoostingClassifier()
gb_clf.fit(default_X_prep, default_Y)

: 

In [ ]:
#Predição sem validação cruzada
pred_train_gb_clf = gb_clf.predict(default_X_prep)

: 

In [ ]:
#Score com predição sem validação cruzada
score_completo(default_Y, pred_train_gb_clf)

: 

In [ ]:
#Predição com validação cruzada
xpred_train_gb_clf = cross_val_predict(gb_clf, default_X_prep, default_Y, cv=5)

: 

In [ ]:
#Score com predição com validação cruzada
score_completo(default_Y, xpred_train_gb_clf)

: 

**Base de Teste**

In [ ]:
#Predição sem validação cruzada
pred_test_gb_clf = gb_clf.predict(default_X_test_prep)

: 

In [ ]:
#Score com predição sem validação cruzada
score_completo(default_Y_test, pred_test_gb_clf)

: 

In [ ]:
#Predição com validação cruzada
xpred_test_gb_clf = cross_val_predict(gb_clf, default_X_prep, default_Y, cv=5)

: 

In [ ]:
#Score com predição com validação cruzada
score_completo(default_Y, xpred_test_gb_clf)

: 

###Modelo após GridSearch

In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid = [
              {'learning_rate': [0.1, 0.05], 'n_estimators': [50, 100], 
               'max_depth': [19, 29]}
              ]
gb_clf_gscv = GradientBoostingClassifier()

grid_search_gb = GridSearchCV(gb_clf_gscv, param_grid, cv=5, return_train_score=True)

grid_search_gb.fit(default_X_prep, default_Y)

: 

In [ ]:
grid_search_gb.best_estimator_

: 

**Base de Treino**

In [ ]:
gb_clf_2 = GradientBoostingClassifier(learning_rate=0.05, max_depth=19)
gb_clf_2.fit(default_X_prep, default_Y)

: 

In [ ]:
#Predição sem validação cruzada
pred_train_gb_clf_2 = gb_clf_2.predict(default_X_prep)

: 

In [ ]:
#Score sem validação cruzada
score_completo(default_Y, pred_train_gb_clf_2)

: 

In [ ]:
#Predição com validação cruzada
xpred_train_gb_clf_2 = cross_val_predict(gb_clf_2, default_X_prep, default_Y, cv=5)

: 

In [ ]:
#Score com validação cruzada
score_completo(default_Y, xpred_train_gb_clf_2)

: 

**Base de Teste**

In [ ]:
#Predição sem validação cruzada
pred_test_gb_clf_2 = gb_clf_2.predict(default_X_test_prep)

: 

In [ ]:
#Score com predição sem validação cruzada
score_completo(default_Y_test, pred_test_gb_clf_2)

: 

In [ ]:
#Predição com validação cruzada
xpred_test_gb_clf_2 = cross_val_predict(gb_clf_2, default_X_test_prep, default_Y_test, cv=5)

: 

In [ ]:
#Score com predição com validação cruzada
score_completo(default_Y_test, xpred_test_gb_clf_2)

: 

#Comparando a performance dos modelos

Para avaliação, usaremos Accuracy, Precision, Recall e F1.

Entendemos, no entanto, que para a caso em estudo o Recall é a métrica mais relevante, pois é importante que identifiquemos o máximo possível de valores positivos na nossa variável alvo, já que os falsos negativos podem levar a impactos financeiros significativos.




**Definições**

**TP:** Verdadeiros Positivos / **FP:** Falsos Positivos / **TN:** Verdadeiros Negativos / **FN:** Falsos Negativos

**Accuracy:** Do total de instâncias, quantas classifiquei corretamente?

$$ Accuracy = \frac{TP + TN}{TP + FP + TN + FN} $$

**Precision:** De todos que classifiquei como positivos, quantos são de fato positivos? 

$$ Precision = \frac{TP}{TP + FP} $$

**Recall:** De todos que são de fato positivos, quantos eu fui capaz de reconhecer? 

$$ Recall = \frac{TP}{TP + FN}$$

**F1:** Média harmônica da Precision e do Recall

$$ F1 = \frac{2}{\frac{1}{Precision}+\frac{1}{Recall}} $$

##Resumo dos resultados

**Resultado na Base de Treino após o GridSearch**

In [ ]:
#Score com validação cruzada 5 fold na base de treino após o GridSearch
score_gs_train_lr = resumo_xval_score(lr, default_X_prep, default_Y, 5)
score_gs_train_bag_lr_2 = resumo_xval_score(bag_lr_2, default_X_prep, default_Y, 5)
score_gs_train_tree_clf_3 = resumo_xval_score(tree_clf3, default_X_prep, default_Y, 5)
score_gs_train_rndf_clf_2 = resumo_xval_score(rndf_clf_2, default_X_prep, default_Y, 5)
score_gs_train_gd_clf_2 = resumo_xval_score(gb_clf_2, default_X_prep, default_Y, 5)

: 

In [ ]:
#DataFrame com todos os resultados no conjunto de treino após o GridSearch
score_gs_train_df = pd.DataFrame(
    np.array([score_gs_train_lr, score_gs_train_bag_lr_2, score_gs_train_tree_clf_3, score_gs_train_rndf_clf_2, score_gs_train_gd_clf_2]),
    index=['LR', 'LR BAG', 'Tree', 'RF', 'GB'],
    columns=['Accuracy', 'Precision', 'Recall', 'F1']
    )
score_gs_train_df

: 

In [ ]:
plt.figure(figsize=(15,15))
plt.suptitle("Resultado na Base de Treino após GridSearch", size='x-large', y=0.93, weight='bold')

plt.subplot(4, 2, 1) # row 1, col 1 index 1
plt.bar(x=score_gs_train_df.index, height=score_gs_train_df['Accuracy'])
plt.title("Accuracy")
plt.ylim([0.7, 1])

plt.subplot(4, 2, 4) # row 4, col 1 index 1
plt.bar(x=score_gs_train_df.index, height=score_gs_train_df['F1'])
plt.title("F1")
plt.ylim([0.7, 1])

plt.subplot(4, 2, 2) # row 2, col 1 index 1
plt.bar(x=score_gs_train_df.index, height=score_gs_train_df['Precision'])
plt.title("Precision")
plt.ylim([0.7, 1])

plt.subplot(4, 2, 3) # row 3, col 1 index 1
plt.bar(x=score_gs_train_df.index, height=score_gs_train_df['Recall'])
plt.title("Recall")
plt.ylim([0.7, 1])

plt.show()

: 

**Resultado na Base de Teste após o GridSearch**

In [ ]:
#Score com validação cruzada 5 fold na base de teste após o GridSearch
score_gs_test_lr = resumo_xval_score(lr, default_X_test_prep, default_Y_test, 5)
score_gs_test_bag_lr_2 = resumo_xval_score(bag_lr_2, default_X_test_prep, default_Y_test, 5)
score_gs_test_tree_clf_3 = resumo_xval_score(tree_clf3, default_X_test_prep, default_Y_test, 5)
score_gs_test_rndf_clf_2 = resumo_xval_score(rndf_clf_2, default_X_test_prep, default_Y_test, 5)
score_gs_test_gd_clf_2 = resumo_xval_score(gb_clf_2, default_X_test_prep, default_Y_test, 5)

: 

In [ ]:
#DataFrame com todos os resultados no conjunto de testes após o GridSearch
score_gs_test_df = pd.DataFrame(
    np.array([score_gs_test_lr, score_gs_test_bag_lr_2, score_gs_test_tree_clf_3, score_gs_test_rndf_clf_2, score_gs_test_gd_clf_2]),
    index=['LR', 'LR BAG', 'Tree', 'RF', 'GB'],
    columns=['Accuracy', 'Precision', 'Recall', 'F1']
    )
score_gs_test_df

: 

In [ ]:
plt.figure(figsize=(15,15))
plt.suptitle("Resultado na Base de Teste após GridSearch", size='x-large', y=0.93, weight='bold')

plt.subplot(4, 2, 1) # row 1, col 1 index 1
plt.bar(x=score_gs_test_df.index, height=score_gs_test_df['Accuracy'])
plt.title("Accuracy")
plt.ylim([0.7, 1])

plt.subplot(4, 2, 4) # row 4, col 1 index 1
plt.bar(x=score_gs_test_df.index, height=score_gs_test_df['F1'])
plt.title("F1")
plt.ylim([0.7, 1])

plt.subplot(4, 2, 2) # row 2, col 1 index 1
plt.bar(x=score_gs_test_df.index, height=score_gs_test_df['Precision'])
plt.title("Precision")
plt.ylim([0.7, 1])

plt.subplot(4, 2, 3) # row 3, col 1 index 1
plt.bar(x=score_gs_test_df.index, height=score_gs_test_df['Recall'])
plt.title("Recall")
plt.ylim([0.7, 1])

plt.show()

: 

##Curva ROC

In [ ]:
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score

: 

In [ ]:
def plot_roc_curve(fpr, tpr, label=None):
    plt.plot(fpr, tpr, linewidth=2, label=label)
    plt.plot([0, 1], [0, 1], 'k--') 
    plt.axis([0, 1, 0, 1])                                    
    plt.xlabel('False Positive Rate (Fall-Out)', fontsize=16) 
    plt.ylabel('True Positive Rate (Recall)', fontsize=16)    
    plt.grid(True)                                            

: 

In [ ]:
fpr_lr, tpr_lr, thresholds_lr = roc_curve(default_Y_test, xpred_test_lr)

: 

In [ ]:
fpr_bag_lr, tpr_bag_lr, thresholds_bag_lr = roc_curve(default_Y_test, cross_pred_test_bag_lr_2)

: 

In [ ]:
fpr_tree, tpr_tree, thresholds_tree = roc_curve(default_Y_test, xpred_test_tree_clf_3)

: 

In [ ]:
fpr_rndf, tpr_rndf, thresholds_rndf = roc_curve(default_Y_test, cross_pred_test_rndf_clf_2)

: 

In [ ]:
fpr_gb, tpr_gb, thresholds_gb = roc_curve(default_Y_test, xpred_test_gb_clf_2)

: 

In [ ]:
plt.figure(figsize=(16, 12))                                    # Not shown
plot_roc_curve(fpr_lr, tpr_lr, label='Regressão Logística')
plot_roc_curve(fpr_bag_lr, tpr_bag_lr, label='Regressão Logística c/ Bagging')
plot_roc_curve(fpr_tree, tpr_tree, label='Árvore de Decisão')
plot_roc_curve(fpr_rndf, tpr_rndf, label='Random Forest')
plot_roc_curve(fpr_gb, tpr_gb, label='Gradient Boosting')
plt.legend(loc="lower right")
plt.title('Curva ROC', fontweight="bold")
plt.show()

: 

#Conclusões

Após os tratamentos dos dados e aplicações dos modelos, utilizamos algumas métricas para avaliar seu desempenho. Dentre elas destacamos Acurácia, Precisão, Recall e F1-Score.

Cabe ressaltar que, para análise de default onde o modelo deverá classificar como 0 (não default) e 1 (como defaut) , destacamos a importância da métrica de Recall, pois a mesma, dentre as situações de Classe Positivo, mostra quantas estão corretas. 

Nessa caso entendemos que Falsos Negativos são mais prejudiciais que os Falsos Positivos, pois classificar como não default (0) quando na verdade seria default, pode gerar grande prejuízo para empresa.

Dentre os modelos utilizados, o que apresentou maior Recall foi o Gradient Boosting (82%), ou seja ele detecta 82% dos default (1). Destacamos que este modelo apresentou a pior performance segundo Accuracy, Precision e F1.

Quando olhamos para métrica F1, que é a média harmônica entre precisão e Recall, modelo que melhor desempenha é o de Random Florest, porém ele leva a uma perda do Recall

Buscando um melhor Recall e com maior F1 possível, o modelo de Gradient Boosting alcança F1 considerável com Recall superior aos demais, mantendo um bom nível de precisão dentre os níveis dos demais modelos.

É importante pontuar que os GridSearch foram executados sem a definição do scoring, portanto buscaram otimizar a Accuracy. Um ponto de melhoria seria rodá-los definindo o *scoring* para Recall ou F1.

Além disso, devido ao tempo de processamento e quantidade de modelos, não foi possível explorar mais opções de parâmetros nos GridSearch, o que também pode ser um ponto de melhoria a ser abordado em um estudo subsequente.

In [ ]:
# Setup plotting
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.callbacks import EarlyStopping

plt.style.use('seaborn-whitegrid')
# Set Matplotlib defaults
plt.rc('figure', autolayout=True)
plt.rc('axes', labelweight='bold', labelsize='large',
       titleweight='bold', titlesize=18, titlepad=10)

: 

In [ ]:
model = keras.Sequential([layers.Dense(1024, activation='relu'),
                          layers.Dropout(0.3),
                          layers.BatchNormalization(),
                          layers.Dense(1024, activation='relu'),
                          layers.Dropout(0.3),
                          layers.BatchNormalization(),
                          layers.Dense(1024, activation='relu'),
                          layers.Dropout(0.3),
                          layers.BatchNormalization(),
                          layers.Dense(1),
                         ])

model.compile(optimizer='adam', loss='mae',)

history = model.fit(default_X_prep, default_Y,
                    validation_data=(default_X_test_prep, default_Y_test),
                    batch_size=256,
                    epochs=100,
                    verbose=0,
                   )

: 

In [ ]:
# Show the learning curves
history_df = pd.DataFrame(history.history)
history_df.loc[:, ['loss', 'val_loss']].plot();

: 

In [ ]:
early_stopping = EarlyStopping(min_delta=0.001, # minimium amount of change to count as an improvement
                               patience=20, # how many epochs to wait before stopping
                               restore_best_weights=True,
                              )

model_stop = keras.Sequential([layers.Dense(512, activation='relu'),
                          layers.Dense(512, activation='relu'),
                          layers.Dense(512, activation='relu'),
                          layers.Dense(1),
                         ])

model_stop.compile(optimizer='adam',loss='mae',)

history2 = model_stop.fit(default_X_prep, default_Y,
                    validation_data=(default_X_test_prep, default_Y_test),
                    batch_size=256,
                    epochs=100,
                    callbacks=[early_stopping],
                    verbose=0,  # turn off training log
                   )

history_df2 = pd.DataFrame(history2.history)
history_df2.loc[:, ['loss', 'val_loss']].plot();
print("Minimum validation loss: {}".format(history_df2['val_loss'].min()))

: 